In [1]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [2]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/prodazha-domov/nizhegorodskaya-oblast?per-page=50'

Добыча колличества ссылок

In [3]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

2495


Составляем массив ссылок

In [4]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 51/51 [00:18<00:00,  2.75it/s]

2427


Составление своей базы данных

In [5]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('a').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Жилая недвижимость'
df['Подсегмент'] = 'Покупка домов/квартир/таунхаусов'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 2427/2427 [11:31<00:00,  3.51it/s]


Посмотрим на результаты вкрадце

In [6]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Объект,Площадь,Площадь участка,...,Газ,Канализация,Ипотека,Гараж,Бассейн,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/prodazha-domov/dom-selo...,"Продажа дома (село Валки, улица Мира, 36)Лыско...",1 300 000 руб.,None,2793786,Лысковский муниципальный,"село Валки, улица Мира, 36",дом,28 /,20 соток,...,NaN,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Покупка домов/квартир/таунхаусов,Нижний Новгород,2022-12-22
1,https://www.gipernn.ru/prodazha-domov/dom-selo...,"Продажа дома (село Валки, улица Мира, 36)Лыско...",,None,2793786,Лысковский муниципальный,"село Валки, улица Мира, 36",дом,28 /,20 соток,...,NaN,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Покупка домов/квартир/таунхаусов,Нижний Новгород,2022-12-22
2,https://www.gipernn.ru/prodazha-domov/dom-selo...,"Продажа дома (село Валки, улица Мира, 36)Лыско...",,None,2793786,Лысковский муниципальный,"село Валки, улица Мира, 36",дом,28 /,20 соток,...,NaN,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Покупка домов/квартир/таунхаусов,Нижний Новгород,2022-12-22
3,https://www.gipernn.ru/prodazha-domov/dom-selo...,"Продажа дома (село Валки, улица Мира, 36)Лыско...",,None,2793786,Лысковский муниципальный,"село Валки, улица Мира, 36",дом,28 /,20 соток,...,NaN,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Покупка домов/квартир/таунхаусов,Нижний Новгород,2022-12-22
4,https://www.gipernn.ru/prodazha-domov/dom-selo...,"Продажа дома (село Валки, улица Мира, 36)Лыско...",,None,2793786,Лысковский муниципальный,"село Валки, улица Мира, 36",дом,28 /,20 соток,...,NaN,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Покупка домов/квартир/таунхаусов,Нижний Новгород,2022-12-22


Экспортируем таблицу

In [7]:
df.to_excel('cottage-buy.xlsx', index=False)